In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
mouths_dict = {
    'января': 'january',
    'февраля': 'february',
    'декабря': 'december',
    'ноября': 'november',
    'октября': 'october',
    'сентября': 'september',
    'мая': 'may',
    'июня': 'june',
    'июля': 'july',
    'августа': 'august',
    'апреля': 'april',
    'марта': 'march'
}
def fix_date(date: str):
    if date:
        date = date.lower()
        for key, value in mouths_dict.items():
            date = date.replace(key, value)
        return date
    return None

In [3]:
fake = faker.Faker(locale='en')

In [4]:
sber_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'Host': 'rabota.sber.ru',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'referer': 'https://rabota.sber.ru/search?keyword='
}

In [5]:
API_URL = "https://rabota.sber.ru/v2/gateway/api/pulseservice/v1/search/get?"
VACANCY_API_URL = "https://rabota.sber.ru/v2/gateway/api/pulseservice/v1/search/get/"

In [13]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip() if x else None

In [6]:
search_params = {
    'page': 12,
    'size': 100
}

In [7]:
first_request = requests.get(API_URL, headers=sber_headers, params=search_params, verify=False)
total_pages = first_request.json()['totalPages']
print(total_pages, first_request.json()['totalCount'])

13 1201


### Получаем списки айдишников со всех страниц

In [36]:
jobs_infos = []
for page in tqdm(range(total_pages)):

    search_params = {
        'page': page,
        'size': 100
    }

    result = requests.get(API_URL, params=search_params, headers=sber_headers, verify=False)
    if result.status_code != 200:
        print(f"Skipping page {page}, response code {result.status_code}")
        continue

    for vacancy in result.json()['vacancies']:
        internal_id = vacancy['internalId']
        jobs_infos.append(internal_id)

    time.sleep(4.0) #sleep for 00 ms

 15%|█▌        | 2/13 [00:08<00:40,  3.70s/it]

Skipping page 1, response code 500


 46%|████▌     | 6/13 [00:28<00:30,  4.36s/it]

Skipping page 5, response code 500


 54%|█████▍    | 7/13 [00:30<00:20,  3.47s/it]

Skipping page 6, response code 500


 77%|███████▋  | 10/13 [00:43<00:11,  3.90s/it]

Skipping page 9, response code 500


100%|██████████| 13/13 [01:04<00:00,  4.99s/it]


In [37]:
len(jobs_infos)

802

In [28]:
jobs_dicts = []
for id in tqdm(jobs_infos):
    result = requests.get(VACANCY_API_URL + str(id), headers=sber_headers, verify=False)
    if result.status_code != 200:
        print(f'error {result.status_code}')
    else:
        try:
            job = result.json()
            job_dict = {
                'title': job['title'],
                'internal_id' : id,
                'url': VACANCY_API_URL + str(id),
                'description': clear_string(job['introduction']),
                'responsibilities': clear_string(job['duties']),
                'qualifications': clear_string(job['requirements']),
                'location': job['location'],
                'company': 'Sber',
                'publish_date': parser.parse(fix_date(job['activeFrom']))
            }
            jobs_dicts.append(job_dict)
        except Exception as e:
            print(f'Error during parsing vacancy {id}: {e}')

    time.sleep(0.5) # 500 ms sleep

 21%|██        | 170/801 [07:12<28:12,  2.68s/it]

error 500


 39%|███▉      | 314/801 [13:41<20:47,  2.56s/it]

error 500


100%|██████████| 801/801 [37:09<00:00,  2.78s/it]  


In [32]:
snapshot = pd.DataFrame(jobs_dicts)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,location,company,publish_date
160,Старший специалист Отдела проверочно-розыскных...,4082971,https://rabota.sber.ru/v2/gateway/api/pulseser...,None,Обязанности: \n \n выезды на место происшестви...,Требования: \n \n образование высшее юридическ...,Город Санкт-Петербург,Sber,2022-06-29 05:23:01
723,Менеджер по продаже зарплатных проектов,4083701,https://rabota.sber.ru/v2/gateway/api/pulseser...,None,Устанавливать первоначальный контакт с партнер...,Высшее образование; \n Опыт успешных продаж от...,"Город Нефтеюганск, Ханты-Мансийский Автономный...",Sber,2022-07-04 05:06:43
632,Специалист банковского сопровождения долевого ...,4081253,https://rabota.sber.ru/v2/gateway/api/pulseser...,None,Обязанности: \n \n Экспертиза документов при п...,Требования: \n \n Высшее образование: экономич...,"Город Екатеринбург, Свердловская Область",Sber,2022-06-27 10:30:23
411,"Водитель-инкассатор, г. Киров Калужской области",4083980,https://rabota.sber.ru/v2/gateway/api/pulseser...,Сбер — лучший работодатель России. У нас более...,Тебе предстоит: \n \n перевозить денежные сред...,Мы ждем от тебя: \n \n образование не ниже сре...,"Город Киров, Кировский Район, Калужская Область",Sber,2022-06-30 12:17:30
295,Java-разработчик (преподаватель),4064061,https://rabota.sber.ru/v2/gateway/api/pulseser...,В Сбере развивается большой проект – IT Bootca...,"Разработка учебных примеров; \n Пилот, ревью, ...",Знание Java/JavaScript от 2-х лет и более; \n ...,Город Москва,Sber,2022-03-10 00:00:00


In [33]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             799 non-null    object        
 1   internal_id       799 non-null    int64         
 2   url               799 non-null    object        
 3   description       495 non-null    object        
 4   responsibilities  799 non-null    object        
 5   qualifications    799 non-null    object        
 6   location          799 non-null    object        
 7   company           799 non-null    object        
 8   publish_date      799 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 56.3+ KB


In [34]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'06-07-2022'

In [35]:
snapshot.to_csv(f'../data/sber/{current_date}.csv')

In [67]:
requests.get(VACANCY_API_URL + str(jobs_infos[2]), headers=sber_headers, verify=False).json()

{'timestamp': '2022-07-05T06:23:43.116+00:00',
 'status': 400,
 'error': 'Bad Request',
 'message': '',
 'path': '/v1/search/get/1848690'}